In [1]:
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Reshape, Conv2DTranspose, BatchNormalization, LeakyReLU
from tensorflow.keras.layers import Activation, Conv2D, Lambda, Concatenate, Flatten, Dense, UpSampling2D

import tensorflow.keras as keras
import colorsys
import tensorflow.keras.backend as K
import glob
import json
from sklearn.model_selection import train_test_split
from io import BytesIO
import PIL
from IPython.display import clear_output, Image, display, HTML

import math
import random

import numpy as np
import matplotlib.pyplot as plt

import cv2

In [2]:
BATCH_SIZE = 32
LATENT_SIZE = 128

In [3]:
def augment(icons):
    aug_icons = []
    for icon in icons:
        for flip in range(4):
            for rotation in range(4):
                aug_icons.append(icon)
                icon = np.rot90(icon)
            icon = np.fliplr(icon)
    return np.asarray(aug_icons)
        

def load_icons(train_size=0.85):
    icon_index = json.load(open('icons/index.json'))
    x = []
    img_rows, img_cols = 32, 32
    for icon in icon_index:
        if icon['name'].endswith('_filled'):
            continue
        img_path = 'icons/png32/%s.png' % icon['name']
        img = load_img(img_path, grayscale=True, target_size=(img_rows, img_cols))
        img = img_to_array(img)
        x.append(img)
    x = np.asarray(x) / 255
    x_train, x_val = train_test_split(x, train_size=train_size)
    return augment(x_train), augment(x_val)

x_train, x_test = load_icons()
x_train.shape, x_test.shape

/home/ohtamans/.pyenv/versions/3.6.7/envs/cookbook/lib/python3.6/site-packages/keras_preprocessing/image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
/home/ohtamans/.pyenv/versions/3.6.7/envs/cookbook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


((83104, 32, 32, 1), (14672, 32, 32, 1))

In [4]:
def create_generative_network(latent_size):
    inp = Input(shape=(latent_size,))
    x = Reshape((1, 1, latent_size))(inp)

    channels = latent_size
    padding = 'valid'
    strides = 1
    for i in range(4):
        x = Conv2DTranspose(channels, kernel_size=4,
                            strides=strides, padding=padding)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(.2)(x)
        
        channels //= 2
        padding = 'same'
        strides = 2

    x = Conv2DTranspose(1, kernel_size=4, strides=1, padding='same')(x)
    image_out = Activation('tanh')(x)

    model = Model(inputs=inp, outputs=image_out)
    return model

generator = create_generative_network(LATENT_SIZE)
generator.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
reshape (Reshape)            (None, 1, 1, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 4, 4, 128)         262272    
_________________________________________________________________
batch_normalization_v2 (Batc (None, 4, 4, 128)         512       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 64)          131136    
_________________________________________________________________
batc

In [5]:
def create_discriminative_network():
    inp = Input(shape=(32, 32, 1))
    x = inp

    channels = 16

    for i in range(4):
        layers = []
        conv = Conv2D(channels, 3, strides=2, padding='same')(x)
        if i:
            conv = BatchNormalization()(conv)
        conv = LeakyReLU(.2)(conv)
        layers.append(conv)
        bv = Lambda(lambda x: K.mean(K.abs(x[:] - K.mean(x, axis=0)), 
                                     axis=-1, 
                                     keepdims=True))(conv)
        layers.append(bv)
        channels *= 2
        x = Concatenate()(layers)

    x = Conv2D(128, 2, padding='valid')(x)
    x = Flatten(name='flatten')(x)
    
    fake = Dense(1, activation='sigmoid', name='generation')(x)

    m = Model(inputs=inp, outputs=fake)
    return m

discriminator = create_discriminative_network()
discriminator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 16)   160         <tensorflow.python.keras.engine.i
__________________________________________________________________________________________________
leaky_re_lu_4 (LeakyReLU)       (None, 16, 16, 16)   0           <tensorflow.python.keras.layers.c
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 16, 16, 1)    0           <tensorflow.python.keras.layers.a
__________________________________________________________________________________________________
concatenat

In [14]:
def gan(g, d):
    # initialize a GAN trainer

    # this is the fastest way to train a GAN in Keras
    # two models are updated simutaneously in one pass

    noise = Input(shape=g.input_shape[1:])
    real_data = Input(shape=d.input_shape[1:])

    generated = g(noise)
    gscore = d(generated)
    rscore = d(real_data)

    def log_eps(i):
        return K.log(i+1e-11)

    # single side label smoothing: replace 1.0 with 0.9
    dloss = - K.mean(log_eps(1-gscore) + .1 * log_eps(1-rscore) + .9 * log_eps(rscore))
    gloss = - K.mean(log_eps(gscore))

    lr, b1 = 1e-4, .2 # otherwise won't converge.
    optimizer = keras.optimizers.Adam(lr, beta1=b1)

    grad_loss_wd = K.gradients(dloss, d.trainable_weights)
    #grad_loss_wd = optimizer.compute_gradients(dloss, d.trainable_weights)
    update_wd = optimizer.apply_gradients(grad_loss_wd)

    grad_loss_wg = optimizer.compute_gradients(gloss, g.trainable_weights)
    update_wg = optimizer.apply_gradients(grad_loss_wg)

    def get_internal_updates(model):
        # get all internal update ops (like moving averages) of a model
        inbound_nodes = model.inbound_nodes
        input_tensors = []
        for ibn in inbound_nodes:
            input_tensors+= ibn.input_tensors
        updates = [model.get_updates_for(i) for i in input_tensors]
        return updates

    other_parameter_updates = [get_internal_updates(m) for m in [d,g]]
    # those updates includes batch norm.

    train_step = [update_wd, update_wg, other_parameter_updates]
    losses = [dloss, gloss]

    learning_phase = K.learning_phase()

    def gan_feed(sess,batch_image,z_input):
        # actual GAN trainer
        nonlocal train_step,losses,noise,real_data,learning_phase

        res = sess.run([train_step,losses],feed_dict={
        noise:z_input,
        real_data:batch_image,
        learning_phase:True,
        # Keras layers needs to know whether
        # this run is training or testring (you know, batch norm and dropout)
        })

        loss_values = res[1]
        return loss_values #[dloss,gloss]

    return gan_feed

print('generating GAN...')
gan_feed = gan(generator, discriminator)

generating GAN...


RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [13]:
keras.backend.gradients?

In [ ]:
def generate_images(generator, count):
    noise = np.random.normal(loc=0., 
                             scale=1., 
                             size=(count, LATENT_SIZE))
    for tile in generator.predict([noise]).reshape((count, 32, 32)):
        tile = (tile * 300).clip(0, 255).astype('uint8')
        yield PIL.Image.fromarray(tile)

def poster(generator, w_count, h_count):
    overview = PIL.Image.new('RGB', (w_count * 34 + 2, h_count * 34 + 2), (128, 128, 128))
    for idx, img in enumerate(generate_images(generator, w_count * h_count)):
        x = idx % w_count
        y = idx // w_count
        overview.paste(img, (x * 34 + 2, y * 34 + 2))
    return overview


In [ ]:
def run(epochs=100):
    sess = K.get_session()
    l = x_train.shape[0]
    l -= l % BATCH_SIZE
    for epoch in range(epochs):
        np.random.shuffle(x_train)
        for batch_start in range(0, l, BATCH_SIZE):
            batch = x_train[batch_start: batch_start + BATCH_SIZE]
            z_input = np.random.normal(loc=0., 
                                       scale=1., 
                                       size=(BATCH_SIZE, LATENT_SIZE))
            dloss, gloss = gan_feed(sess, batch, z_input)
        clear_output(wait=True)
        print('%d dloss: %2.2f gloss: %2.2f' % (epoch, dloss, gloss))
        f = BytesIO()
        poster(generator, 8, 5).save(f, 'png')
        display(Image(data=f.getvalue()))
run(1)

In [ ]:
run(30)

In [ ]:
f = BytesIO()
poster(generator, 24, 12).save(f, 'png')
display(Image(data=f.getvalue()))